## How To Perform an Amplitude Rabi Experiment

### Prerequisites
This guide assumes you have a configured `DeviceSetup` as well as `Qubit` objects with assigned parameters. Please see these guides (add links) if you need to create your setup and qubits for the first time. However, you can also run this notebook "as is" using an emulated session. If you are just getting started with the LabOne Q Applications Library, please don't hesitate to reach out to us at info@zhinst.com.

### Background
In this how-to guide, you'll perform a measurement to find the optimal pulse amplidue to induce a $\pi$-rotation for a given qubit transition using the `amplitude_rabi` experiment included in the LabOne Q Applications Library. In an amplitude-Rabi experiment, you'll vary the drive-pulse amplitude, $A$, and measure the state of the qubit as it rotates around the Bloch sphere depending on $A$, as depicted below. 

![](../images/rabi_pulses_and_sphere.svg "Demonstration of pulses and the corresponding rotation around the Bloch sphere during an amplitude Rabi experiment.")

For a qubit oscialling between the ground state, $\ket{0}$, and excited state, $\ket{1}$, the population behaves depends on the drive-pulse amplitude according to:

$$\rho(\ket{1}) = sin^2 \bigg(\int_{0}^{\tau}\frac{\Omega_{R}(A)}{2}dt \bigg) = sin^2\bigg(\frac{\theta}{2}\bigg)$$

This results in data that will, ignoring any other effects, look like:

![](<../images/rabi_plot.svg> "Plot of Rabi oscillations with the population of the excited state on the left axis and the pulse amplitude on the bottom axis. Measurement points are depicted as cyan circles.")

### Imports

You'll start by importing the amplitude-Rabi experiment from `laboneq_applications`, as well as `laboneq.simple` and a demo QPU and device setup to run in emulation mode.

In [1]:
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation
from laboneq.simple import *

from laboneq_applications.experiments import amplitude_rabi
from laboneq_applications.qpu_types.tunable_transmon.demo_qpus import demo_platform

### QPU and Device Setup

You'll generate six qubits with pre-defined parameters, as well as a `Device_Setup` consisting of a SHFQC+, HDAWG, and PQSC. If you already have your own `DeviceSetup` and qubits configured, you'll instead initialize the session using your setup.

In [2]:
my_platform = demo_platform(6)

Then, you'll connect to the `Session`. Here we connect to an emulated one:

In [ ]:
session = Session(my_platform.setup)
session.connect(do_emulation=True)

### Running the Rabi Workflow

You'll now make the experiment workflow and run:

In [ ]:
# our qubits live here in the demo setup:
qubits = my_platform.qpu.qubits

default_opts = amplitude_rabi.experiment_workflow.options()
default_opts.do_analysis(False)

my_workflow = amplitude_rabi.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubits=[qubits[0], qubits[1]],
    amplitudes=[[0, 0.25, 0.5, 0.75, 1], [0, 0.25, 0.5, 0.75, 1]],
    options=default_opts,
)

my_results = my_workflow.run()

#### Output Simulation

You can also inspect the compiled experiment and plot the simulated output:

In [ ]:
compiled_rabi = my_results.tasks["compile_experiment"].output
plot_simulation(compiled_rabi, length=50e-6)

#### Inspecting the Source Code

You can inspect the source code of the `create_experiment` task defined in `amplitude_rabi` to see how the experiment pulse sequence is created:

In [ ]:
amplitude_rabi.create_experiment.src

### Changing the Options

We can give our Rabi experiment options. First, inspect what they currently are:

In [ ]:
my_new_opts = amplitude_rabi.experiment_workflow.options()
my_new_opts

Then provide new options. This time, you'll remove the calibration traces and change the counts.

In [ ]:
my_new_opts.count(2048)
my_new_opts.use_cal_traces(value=False)
my_new_opts.do_analysis(False)
my_new_opts

#### Run the workflow with updated options

Now, run the workflow with new options and inspect the simulated output. You'll notice that the calibration traces are no longer there.

In [ ]:
my_new_workflow = amplitude_rabi.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubits=[qubits[0], qubits[1]],
    amplitudes=[[0, 0.25, 0.5, 0.75, 1], [0, 0.25, 0.5, 0.75, 1]],
    options=my_new_opts,
)

my_new_results = my_new_workflow.run()
new_compiled_rabi = my_new_results.tasks["compile_experiment"].output
plot_simulation(new_compiled_rabi, length=50e-6)

### Temporary settings

The qubit parameters are used to control the settings of pulses and instruments during the experiment. We can run the amplitude-Rabi experiment with different settings by passing it a copy of the qubits with modified parameters:

In [ ]:
# Make a copy of the qubits
temp_qubits = my_platform.qpu.copy_qubits()
# Change the length of the drive pulses
temp_qubits[0].parameters.ge_drive_length = 1000e-9
temp_qubits[1].parameters.ge_drive_length = 1000e-9

my_new_workflow = amplitude_rabi.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubits=[temp_qubits[0], temp_qubits[1]],  # pass temporary qubits
    amplitudes=[[0, 0.25, 0.5, 0.75, 1], [0, 0.25, 0.5, 0.75, 1]],
    options=my_new_opts,
)

my_new_results = my_new_workflow.run()
new_compiled_rabi = my_new_results.tasks["compile_experiment"].output
plot_simulation(new_compiled_rabi, length=10e-6)

Great! You've now run your Rabi experiment. Check out these other experiments to keep characterizing your qubits:

In [ ]:
# TODO: Add experiment links
# TODO: Add Analysis